In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, random_split, DataLoader
import torch.nn as nn
import os
import zipfile
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix,f1_score, accuracy_score
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv("/content/drive/MyDrive/RELIANCE.csv")
df = df.dropna()
numeric_columns = df.select_dtypes(include=[float, int]).columns
max_vwap = df['VWAP'].max()

# Apply max normalization to numeric columns only
normalized_df = df.copy()
normalized_df[numeric_columns] = df[numeric_columns].apply(lambda x: x / x.max())

print(df)
normalized_df

            Date    Symbol Series  Prev Close     Open     High      Low  \
2850  2011-06-01  RELIANCE     EQ      951.85   952.00   958.65   943.65   
2851  2011-06-02  RELIANCE     EQ      946.80   936.55   954.70   936.55   
2852  2011-06-03  RELIANCE     EQ      951.05   960.50   967.00   931.50   
2853  2011-06-06  RELIANCE     EQ      934.60   934.65   940.80   928.15   
2854  2011-06-07  RELIANCE     EQ      937.75   933.55   960.00   933.55   
...          ...       ...    ...         ...      ...      ...      ...   
5301  2021-04-26  RELIANCE     EQ     1904.35  1920.00  1962.00  1911.50   
5302  2021-04-27  RELIANCE     EQ     1937.85  1940.00  1997.20  1938.25   
5303  2021-04-28  RELIANCE     EQ     1988.65  1997.85  2008.00  1980.15   
5304  2021-04-29  RELIANCE     EQ     1997.30  2022.90  2044.50  2007.30   
5305  2021-04-30  RELIANCE     EQ     2024.05  2008.50  2036.00  1987.55   

         Last    Close     VWAP   Volume      Turnover    Trades  \
2850   947.50   946

,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
2850,2011-06-01,RELIANCE,EQ,0.409477,0.409462,0.404605,0.408409,0.407544,0.407305,0.405149,0.028184,0.011826,0.041043,0.046961,0.588807
2851,2011-06-02,RELIANCE,EQ,0.407305,0.402817,0.402938,0.405336,0.409695,0.409133,0.404833,0.033005,0.013839,0.044145,0.055575,0.595052
2852,2011-06-03,RELIANCE,EQ,0.409133,0.413118,0.408129,0.403151,0.402598,0.402056,0.406799,0.066966,0.028215,0.090154,0.053961,0.284736
2853,2011-06-06,RELIANCE,EQ,0.402056,0.402000,0.397071,0.401701,0.403716,0.403411,0.399789,0.021550,0.008923,0.030371,0.024831,0.407229
2854,2011-06-07,RELIANCE,EQ,0.403411,0.401527,0.405174,0.404038,0.412749,0.412230,0.406312,0.061718,0.025972,0.062096,0.126332,0.723310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5301,2021-04-26,RELIANCE,EQ,0.819234,0.825806,0.828075,0.827292,0.833584,0.833645,0.829815,0.147488,0.126758,0.181406,0.222802,0.533806
5302,2021-04-27,RELIANCE,EQ,0.833645,0.834409,0.842932,0.838870,0.855951,0.855499,0.845768,0.141444,0.123901,0.203850,0.196516,0.490933
5303,2021-04-28,RELIANCE,EQ,0.855499,0.859290,0.847490,0.857004,0.857306,0.859220,0.853872,0.121139,0.107131,0.173142,0.204301,0.596013
5304,2021-04-29,RELIANCE,EQ,0.859220,0.870065,0.862895,0.868754,0.868855,0.870728,0.865247,0.123192,0.110398,0.149216,0.147648,0.423562


In [26]:
class Stocks_Data(Dataset):

    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        data = []

        if index > (len(self.df) - 21):
          for i in range(index - 20, index):
            slice1 = self.df.iloc[i][4 : 11]
            app = slice1.values.astype(float)
            data.append(app)
          label = self.df.iloc[index][9]
          # if label[0]
          data = torch.tensor(data)

        else :
          for i in range(index, index + 20):
            slice1 = self.df.iloc[i][4 : 11]
            app = slice1.values.astype(float)
            data.append(app)
          label = self.df.iloc[index + 20][9]
          # label = torch.tensor(label.values.astype(float))
          data = torch.tensor(data)

        return data, label

In [27]:
batch_size = 12
ds = Stocks_Data(normalized_df)

print(len(ds))
train_size = int(0.9 * len(ds))
test_size = len(ds) - train_size
train_dataset, test_dataset = random_split(ds, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

2456


In [28]:
for data, label in train_loader:
  print(data.size())
  print(label)
  print(label.size())
  break

torch.Size([12, 20, 7])
tensor([0.3757, 0.3517, 0.4584, 0.3769, 0.6544, 0.4153, 0.4357, 0.3855, 0.3124,
        0.3679, 0.4251, 0.6706], dtype=torch.float64)
torch.Size([12])


In [29]:
class Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, device):
        super(Model,self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.device = device
        # self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.self_attention = nn.MultiheadAttention(hidden_size, num_heads = 64, batch_first = True)
        self.fc = nn.Sequential(nn.Linear(input_size, hidden_size))

        self.fc2 = nn.Sequential(nn.Linear(hidden_size, 1))

        self.sigmoid = nn.Sigmoid()

    def forward(self, lead_data):
        batch_size = lead_data.size(0)
        seq_length = lead_data.size(1)
        # h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(self.device)
        # h0 = h0.double()
        # c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(self.device)
        # c0 = c0.double()
        # lead_data = lead_data.double()
        # lstm_out, _ = self.lstm(lead_data, (h0, c0))

        # data = self.flat(lead_data)

        #lead_data = lead_data.permute(1,0,2)
        lead_data = self.fc(lead_data)
        attention_out,_ = self.self_attention(lead_data,lead_data,lead_data)
        #attention_out = attention_out.permute(1,0,2)
        last_output = attention_out[:, -1, :]

        output = self.fc2(last_output)
        # output = self.fc2(data)

        return output
input_size = 7
num_layers = 4
hidden_size = 128
model = Model(input_size,hidden_size,num_layers, device).double().to(device)
try:
  model.load_state_dict(torch.load("/content/drive/MyDrive/Stock_lstm.pth"))
except:
  print("No Saved State Found")

No Saved State Found


In [35]:
val_losses = []
train_losses = []
epochs = 100
criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0001)

for epoch in range(epochs):
    train_loss = 0.0
    val_loss = 0.0


    model.train()
    for i, (data, label) in enumerate(tqdm(train_loader)):

        # label = F.one_hot(label, num_classes=2)
        label = label.double()
        label = label.unsqueeze(1)
        label = label.to(device)

        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * data.size(0)
        # if i % 100 == 0:
        #   print("EPOCH : {}/{}, MIN_LOSS : {}, LOSS : {}".format(epoch + 1, epochs, min_loss, loss.item()))
          # print("EPOCH : {}/{}, LOSS : {}".format(epoch + 1, epochs, loss.item()))

    train_loss /= i
    train_losses.append(train_loss)

    # Validation
    # model.eval()
    for data, label in train_loader:
      # print(label)
      # print(model(data.to(device)))
      print(train_loss, epoch + 1)
      break

100%|██████████| 185/185 [00:11<00:00, 16.11it/s]


0.003779987354703261 1


100%|██████████| 185/185 [00:11<00:00, 16.50it/s]


0.0035085978120059096 2


100%|██████████| 185/185 [00:09<00:00, 19.36it/s]


...

0.0030735194729837882 98


100%|██████████| 185/185 [00:10<00:00, 18.47it/s]


0.0029825732756902303 99


100%|██████████| 185/185 [00:10<00:00, 16.98it/s]


0.002963303621982749 100


In [37]:
torch.save(model.state_dict(), "/content/drive/MyDrive/Stock_attention.pth")

In [36]:
for data, labels in train_loader:
  outputs1 = model(data.to(device)) * max_vwap
  label = labels * max_vwap
  for i in range(batch_size):
    print("original - ", int(label[i].item()), end = " \t")
    print("attention - ", int(outputs1[i].item()))
  break

original -  1040 	attention -  1030
original -  1464 	attention -  1425
original -  1318 	attention -  1303
original -  1098 	attention -  1092
original -  976 	attention -  972
original -  1003 	attention -  1013
original -  933 	attention -  964
original -  733 	attention -  768
original -  827 	attention -  807
original -  1083 	attention -  1099
original -  1352 	attention -  1343
original -  970 	attention -  929
